In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Load IMDb dataset from TensorFlow datasets
imdb = tf.keras.datasets.imdb
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")
print(f"Example review (tokenized): {X_train[0]}")
print(f"Sentiment (0=negative, 1=positive): {y_train[0]}")

# Decode reviews for understanding
word_index = imdb.get_word_index()
reverse_word_index = {v: k for k, v in word_index.items()}
decoded_review = " ".join([reverse_word_index.get(i - 3, "?") for i in X_train[0]])
print(f"Decoded review: {decoded_review}")


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training samples: 25000, Testing samples: 25000
Example review (tokenized): [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 

In [ ]:
max_length = 200
X_train_padded = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test, maxlen=max_length, padding='post')


In [ ]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=max_length),
    LSTM(64, return_sequences=True),
    Dropout(0.5),
    LSTM(32),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
history = model.fit(
    X_train_padded, y_train,
    epochs=5,
    batch_size=128,
    validation_split=0.2,
    verbose=1
)


Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 83s 496ms/step - accuracy: 0.5188 - loss: 0.6922 - val_accuracy: 0.5980 - val_loss: 0.6991
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 80s 484ms/step - accuracy: 0.6120 - loss: 0.6354 - val_accuracy: 0.6570 - val_loss: 0.5800
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 87s 516ms/step - accuracy: 0.7174 - loss: 0.5092 - val_accuracy: 0.8202 - val_loss: 0.4570
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 81s 513ms/step - accuracy: 0.8436 - loss: 0.4191 - val_accuracy: 0.6852 - val_loss: 0.6590
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 76s 486ms/step - accuracy: 0.7192 - loss: 0.5387 - val_accuracy: 0.7894 - val_loss: 0.4954


In [ ]:
# Evaluate on test data
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


782/782 ━━━━━━━━━━━━━━━━━━━━ 38s 48ms/step - accuracy: 0.7905 - loss: 0.4933
Test Accuracy: 79.34%


In [ ]:
model.save("sentiment_analysis_model.h5")


In [ ]:
def preprocess_text(text):
    tokens = [word_index.get(word, 2) for word in text.lower().split()]  # Convert words to indexes
    padded_tokens = pad_sequences([tokens], maxlen=max_length, padding='post')
    return padded_tokens

test_review = "This movie was fantastic! The acting was great and the plot was intriguing."
test_padded = preprocess_text(test_review)
predicted_sentiment = model.predict(test_padded)
print("Positive" if predicted_sentiment > 0.5 else "Negative")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
Positive


In [ ]:
# Sample negative review
test_review_negative = "This movie was boring and the plot was predictable."

def preprocess_text(text):
    tokens = [word_index.get(word, 2) for word in text.lower().split()]
    padded_tokens = pad_sequences([tokens], maxlen=max_length, padding='post')
    return padded_tokens

test_padded_negative = preprocess_text(test_review_negative)

# Predict sentiment using the model
predicted_sentiment_negative = model.predict(test_padded_negative)

if predicted_sentiment_negative > 0.5:
    print("Positive")
else:
    print("Negative")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Positive


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

test_review_negative = "This movie was boring and the plot was predictable."

def preprocess_text(text):
    tokens = [word_index.get(word, 2) for word in text.lower().split()]
    padded_tokens = pad_sequences([tokens], maxlen=max_length, padding='post')
    return padded_tokens

test_padded_negative = preprocess_text(test_review_negative)

predicted_sentiment_negative = model.predict(test_padded_negative)

print(f"Prediction Probability: {predicted_sentiment_negative[0][0]}")
threshold = 0.5
if predicted_sentiment_negative > threshold:
    print("Positive")
else:
    print("Negative")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediction Probability: 0.7853353023529053
Positive
